<a href="https://colab.research.google.com/github/illliyas/Machine-Learning/blob/master/tweets/Live_Coding%20-%20Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install bertopic
!pip install joblib==1.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd # reading tweets
import re # for cleaning
import nltk # NLP
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bertopic import BERTopic #topic modelling

In [5]:
df = pd.read_csv('ek_tweets.csv')

In [6]:
df.head()

,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet
0,3.278147e+09,2022-10-06 01:48:35+00:00,,1.580000e+18,en,0,0,0,0,Twitter for Android,"@EmiratesSupport Thanks for the support, hope ..."
1,1.304480e+08,2022-10-06 02:34:23+00:00,,1.580000e+18,en,0,0,1,0,Twitter for iPhone,@EmiratesSupport Have replied to message. Plea...
2,1.693349e+08,2022-10-06 04:37:50+00:00,,1.580000e+18,en,0,0,0,0,Twitter Web App,@EmiratesSupport @EmiratesSupport i have DM'd ...
3,4.855336e+08,2022-10-06 04:41:22+00:00,,1.580000e+18,en,0,0,1,0,Twitter for iPhone,@EmiratesSupport sent you a dm Janine
4,6.556422e+07,2022-10-06 06:29:02+00:00,,1.580000e+18,en,0,0,1,0,Twitter for iPhone,@EmiratesSupport I have been mailing for fligh...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   author id      934 non-null    float64
 1   created_at     934 non-null    object 
 2   geo            934 non-null    object 
 3   id             934 non-null    float64
 4   lang           934 non-null    object 
 5   like_count     934 non-null    int64  
 6   quote_count    934 non-null    int64  
 7   reply_count    934 non-null    int64  
 8   retweet_count  934 non-null    int64  
 9   source         934 non-null    object 
 10  tweet          934 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 80.4+ KB


In [8]:
df = df.drop_duplicates(subset='tweet', keep="first")

In [10]:
list(df['tweet'][106:120])

['@EmiratesSupport I booked my flight from HYD-EWR for 3rd Nov using Student offer... I just wanted to know if I can merge the extra 10kg baggage allowance as per student offer in the luggage itself or do I need to carry an extra baggage of 10kg separately. Please reply!',
 '@EmiratesSupport I have shared details in ur DM',
 '@EmiratesSupport I have now given up.  Possibly easy to take this to a small claims court when I return to Australia',
 '@EmiratesSupport Finally, almost 45 days after my bag was broken, I have an answer: please, wait 20 days more for a "personalise response". Seriously? @EmiratesSupport @emirates @facua https://t.co/UQmSUTN443',
 '@EmiratesSupport See you this Friday for the return trip back. 🙂',
 'RT @fatman73: @EmiratesSupport @emirates @EmiratesSupport ticket cancelled on 07th Oct. Still no word from @emirates on refund amount and r…',
 '@EmiratesSupport The reply is useless I cannot submit the form. I paid over £3000 for my ticket and was delayed over 24 hour

In [11]:
def cleaning_tweets(tweet):
    temp = tweet.lower()
    #Removing hashtags and mentions
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    #Removing links
    temp = re.sub(r"http\S+", "", temp)
    temp = re.sub(r"www.\S+", "", temp)
    #Removing punctuations
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    #Filtering non-alphanumeric characters
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = re.sub("([a-z]*[0-9]+[a-z0-9]* )+"," ", temp)
    #remove 2 letter words
    temp = re.sub(r"\b[a-z]{2}\b"," ", temp)
    return temp

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
sw = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [17]:
def remove_stopwords(tweet):
  words = word_tokenize(tweet)
  words = [word for word in words if word not in sw]
  words = [lemmatizer.lemmatize(word) for word in words]
  return ' '.join(words)

In [18]:
def number_of_words(tweet):
  return len(word_tokenize(tweet))

In [24]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [25]:
df['tweet_clean'] = df.apply(lambda row: cleaning_tweets(row['tweet']), axis=1)
df['tweet_clean2'] = df.apply(lambda row: remove_stopwords(row['tweet_clean']), axis=1)
df['number_of_words'] = df.apply(lambda row: number_of_words(row['tweet_clean2']), axis=1)

In [27]:
list(df.tweet)[106:120]

['@EmiratesSupport I booked my flight from HYD-EWR for 3rd Nov using Student offer... I just wanted to know if I can merge the extra 10kg baggage allowance as per student offer in the luggage itself or do I need to carry an extra baggage of 10kg separately. Please reply!',
 '@EmiratesSupport I have shared details in ur DM',
 '@EmiratesSupport I have now given up.  Possibly easy to take this to a small claims court when I return to Australia',
 '@EmiratesSupport Finally, almost 45 days after my bag was broken, I have an answer: please, wait 20 days more for a "personalise response". Seriously? @EmiratesSupport @emirates @facua https://t.co/UQmSUTN443',
 '@EmiratesSupport See you this Friday for the return trip back. 🙂',
 'RT @fatman73: @EmiratesSupport @emirates @EmiratesSupport ticket cancelled on 07th Oct. Still no word from @emirates on refund amount and r…',
 '@EmiratesSupport The reply is useless I cannot submit the form. I paid over £3000 for my ticket and was delayed over 24 hour

In [28]:
list(df.tweet_clean2)[106:120]

['booked flight hyd ewr nov using student offer wanted know merge extra baggage allowance per student offer luggage need carry extra baggage separately please reply',
 'shared detail',
 'given possibly easy take small claim court return australia',
 'finally almost day bag broken answer please wait day personalise response seriously',
 'see friday return trip back',
 'ticket cancelled oct still word refund amount r',
 'reply useless submit form paid ticket delayed hour character acceptable',
 'asked fragile sticker ended despite official complaint follow day resolution timeline even single follow call way expectation',
 'day since luggage missing actually missing located month ago instead sent kabul funny story one want take responsibility mistake',
 'submitted complaint since start october chance someone could respond email',
 'partner visa uaq ftz new time limit stay uae visa holder',
 'tried join skywards program filled name incorrectly change told use live chat trying almost month 

In [30]:
df2 = df[df['number_of_words']>2]

In [31]:
tweets = df2.tweet_clean2.to_list()

In [ ]:
#Topic Modelling

In [33]:
model = BERTopic(verbose=True, embedding_model= 'paraphrase-MiniLM-L3-v2' , min_topic_size=6)

In [34]:
topics, _ = model.fit_transform(tweets)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/27 [00:00<?, ?it/s]

2022-11-23 06:36:16,514 - BERTopic - Transformed documents to Embeddings
2022-11-23 06:36:29,817 - BERTopic - Reduced dimensionality
2022-11-23 06:36:29,895 - BERTopic - Clustered reduced embeddings


In [35]:
model.get_topic_info()

,Topic,Count,Name
0,-1,280,-1_flight_please_service_help
1,0,54,0_dubai_visa_uae_flight
2,1,36,1_complaint_email_response_nothing
3,2,35,2_seat_baby_selection_live
4,3,29,3_airline_experience_ever_amp
5,4,28,4_bag_baggage_damaged_bali
6,5,26,5_luggage_baggage_checked_carry
7,6,21,6_lounge_dublin_layover_hotel
8,7,21,7_cancel_flight_cancellation_cancelled
9,8,20,8_book_trying_technical_flight


In [36]:
model.visualize_barchart()

In [37]:
model.visualize_hierarchy()

In [38]:
model.visualize_topics()

In [39]:
df2['tweet_topics'] = topics

In [40]:
df2.head()

,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,tweet_clean,tweet_clean2,number_of_words,tweet_topics
0,3.278147e+09,2022-10-06 01:48:35+00:00,,1.580000e+18,en,0,0,0,0,Twitter for Android,"@EmiratesSupport Thanks for the support, hope ...",thanks for the support hope will get resol...,thanks support hope get resolved asap,6,25
1,1.304480e+08,2022-10-06 02:34:23+00:00,,1.580000e+18,en,0,0,1,0,Twitter for iPhone,@EmiratesSupport Have replied to message. Plea...,have replied message please check,replied message please check,4,20
2,1.693349e+08,2022-10-06 04:37:50+00:00,,1.580000e+18,en,0,0,0,0,Twitter Web App,@EmiratesSupport @EmiratesSupport i have DM'd ...,i have d u but one seems reply can ...,u one seems reply u please help regard,8,-1
4,6.556422e+07,2022-10-06 06:29:02+00:00,,1.580000e+18,en,0,0,1,0,Twitter for iPhone,@EmiratesSupport I have been mailing for fligh...,i have been mailing for flight delay compensa...,mailing flight delay compensation one respondi...,13,-1
5,8.179550e+08,2022-10-06 09:48:29+00:00,,1.580000e+18,en,0,0,1,0,Twitter Web App,"@EmiratesSupport Hi, we are flying economy fro...",are flying economy from lgw dxb oct...,flying economy lgw dxb october luggage allowan...,20,2


In [41]:
topic_df = pd.DataFrame(model.topic_labels_.items(),columns = ['topic_id', 'topics_name'])

In [42]:
topic_df

,topic_id,topics_name
0,-1,-1_flight_please_service_help
1,0,0_dubai_visa_uae_flight
2,1,1_complaint_email_response_nothing
3,2,2_seat_baby_selection_live
4,3,3_airline_experience_ever_amp
5,4,4_bag_baggage_damaged_bali
6,5,5_luggage_baggage_checked_carry
7,6,6_lounge_dublin_layover_hotel
8,7,7_cancel_flight_cancellation_cancelled
9,8,8_book_trying_technical_flight


In [43]:
df3 = df2.merge(topic_df,how='left',left_on='tweet_topics',right_on='topic_id')

In [47]:
list(df3[df3['topics_name']=='13_nut_meal_food_allergy']['tweet'])

['@EmiratesSupport I am vegetarian. What meal option should I choose. Don’t particularly want Indian as spicy.',
 '@EmiratesSupport @emirates can you please offer a solution, as I have paid for a meal and would like to enjoy one. https://t.co/5EZikn7T6D',
 '@EmiratesSupport please don’t leave my special meal in London for tomorrows flight like you did back in May and I was stuck with just lettuce for my dinner😭',
 '@EmiratesSupport hello we’re looking to fly to SA and Emirates has suitable flights. My son has a nut allergy and carries epipens. Do you still not withdraw the sale of nuts during your flights? If this is the case we will fly with another airline.',
 'RT @AbbyHants: @EmiratesSupport hello we’re looking to fly to SA and Emirates has suitable flights. My son has a nut allergy and carries ep…',
 '@EmiratesSupport Reply: We serve nuts on all our flights, either as a meal ingredient or as an accompaniment to drinks. Other passengers may bring food on board that contains nuts, an